In this notebook, Pu will pratice his Pytorch skills by writing a regression algorithm.

In [25]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv(url, header=None, delimiter=r'\s+', names=column_names)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data (important for regression)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert into Pytorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create TensorDataset from our data
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [26]:
import torch.nn as nn

# Define the model
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 16)
        self.layer2 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        return self.layer2(x)

In [27]:
# Hyperparameters
learning_rate = 0.01
epochs = 5000

input_dim = X_train.shape[1]
model = ANN(input_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Training Loop
model.train()
for epoch in range(epochs):
    total_train_loss = 0
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Forward
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    if (epoch + 1) % 500 == 0:
        average_train_loss = total_train_loss / len(train_loader)
        print(f'Epoch [{epoch + 1} / {epochs}], Loss: {average_train_loss:.4f}')

# Evaluation
model.eval()
total_test_loss = 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        y_pred = model(x_batch)
        test_loss = criterion(y_pred, y_batch)
        total_test_loss += test_loss.item()

average_test_loss = total_test_loss / len(test_loader)
print(f'Average Test MSE: {average_test_loss:.4f}')

Epoch [500 / 5000], Loss: 24.0963
Epoch [1000 / 5000], Loss: 21.8610
Epoch [1500 / 5000], Loss: 21.6906
Epoch [2000 / 5000], Loss: 20.6525
Epoch [2500 / 5000], Loss: 17.7836
Epoch [3000 / 5000], Loss: 21.2120
Epoch [3500 / 5000], Loss: 17.5638
Epoch [4000 / 5000], Loss: 18.6689
Epoch [4500 / 5000], Loss: 16.8521
Epoch [5000 / 5000], Loss: 18.8670
Average Test MSE: 20.6977
